In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from textblob import TextBlob
import re

In [3]:
def remove_mentions(tweet):
    return re.sub('@[A-Za-z0-9_]+'," ",tweet)

def remove_hashtags(tweet): 
    return re.sub('#[A-Za-z0-9_]+'," ",tweet)

def remove_links(tweet): 
    tweet = re.sub('[()!?]'," ",tweet)           
    tweet = re.sub(r'http\S+'," ",tweet)         
    tweet = re.sub(r'www.\S+'," ",tweet)         
    tweet = re.sub(r'\[.*?\]',' ', tweet)        
    return re.sub(r'[^a-z0-9]'," ",tweet)         

def get_sentiment(tweet):
    blob = TextBlob(tweet)
    return blob.sentiment.polarity

def get_sentiment_name(sentiment):
    if sentiment == 0:
        return "Neutral ðŸ˜Š"
    elif sentiment > 0:
        return "PositiveðŸ¥³"
    else:
        return "NegativeðŸ¤¬"

def load_data():
    df = pd.read_csv('Valorant.csv')
    df.rename(columns={'Text':'Tweets'}, inplace = True)
    df['Tweets'] = df['Tweets'].str.lower()
    df['Tweets'] = df['Tweets'].apply(remove_hashtags)
    df['Tweets'] = df['Tweets'].apply(remove_mentions)
    df['Tweets'] = df['Tweets'].apply(remove_links)
    df['Sentiment'] = df['Tweets'].apply(get_sentiment)
    df['SentimentName'] = df["Sentiment"].apply(get_sentiment_name)
    return df

In [19]:
df = load_data()
counter = df.SentimentName.reset_index()
counter['Date'] = df['Date']
# counter[counter['Date']<='2020-02-04']
counter = pd.concat([counter,pd.get_dummies(counter['SentimentName'])], axis=1)
counter['Date'] = pd.to_datetime(counter['Date'])
counterdf = counter.set_index('Date').resample('D').sum()
counterdf.replace(0,np.nan,inplace=True)
counterdf.dropna(how='all',inplace=True)
# fig = px.funnel(counterdf, counterdf.index, ['NegativeðŸ¤¬','Neutral ðŸ˜Š','PositiveðŸ¥³'])
fig = px.funnel(counterdf, counterdf.index, ['NegativeðŸ¤¬','Neutral ðŸ˜Š','PositiveðŸ¥³'],color='variable',color_discrete_sequence=["red", "yellow", "green"])
fig

In [16]:
vdf = load_data()
count = vdf.SentimentName.value_counts().reset_index()
fig = px.bar(count,'index','SentimentName',color='index',color_discrete_sequence=[
                  "yellow", "green", "red"])
fig